# Backcasting Demo Notebook

_Loren Champlin_

Adapted from _Adarsh Pyarelal_'s WM 12 Month Evaluation Notebook 

As always, we begin with imports, and print out the commit hash for a rendered
version of the notebook.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
import numpy as np
import pandas as pd
from delphi.db import engine
import random as rm
import delphi.evaluation_port as EN
import warnings
warnings.filterwarnings("ignore")
import logging
logging.getLogger().setLevel(logging.CRITICAL)
from delphi.cpp.DelphiPython import AnalysisGraph as AG, InitialBeta as IB, RNG
import time
import seaborn as sns
import matplotlib.pyplot as plt

Here I will set random seeds

In [ ]:
np.random.seed(87)
rm.seed(87)

In [ ]:
G = AG.from_json_file("../scripts/data/raw/wm_12_month_evaluation/jsonld-merged20190404.json",0.7,0.5)

In [ ]:
nodes_to_remove = {
    "UN/events/human/physical_insecurity",
    "UN/entities/human/government/government_actions/duty",
    "UN/entities/human/livelihood",
    "UN/entities/GPE",
    "UN/events/crisis",
    "UN/entities/natural/biology/ecosystem",
    "UN/entities/natural/crop_technology/management",
    "UN/events/nature_impact/climate_change_mitigation",
    "UN/entities/food_availability",
    "UN/entities/human/food/food_insecurity",
    "UN/entities/human/financial/economic/fuel"
}



In [ ]:
G.remove_nodes(nodes_to_remove)


In [ ]:
G.print_indicators()
print("\n")

In [ ]:
G.get_subgraph_for_concept("UN/events/human/human_migration",2,True)
#G.prune(cutoff=2)

In [ ]:
G.print_indicators()

In [ ]:
G.to_png()

In [ ]:
query = " ".join(
        [
            f"select * from indicator",
            f"where `Variable` like 'New asylum seeking applicants'",
        ]
    )

results = engine.execute(query)

df_results = pd.DataFrame(results,columns=results.keys())
df_results